### import Module

In [1]:
import torch
from einops import rearrange
from einops.layers.torch import Rearrange
import numpy as np

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 3090


### Make Patches

In [22]:
import torch.nn as nn
## input image batch 1000개의 Sample RGB 3개의 채널 height 244 X  width 244 크기이다. 
x = torch.randn(1000, 3, 224, 224)

## Image를 이미지의 크기/패치의 크기의 패치로 쪼개고 Flatten 시킨다. 
p = 16 # 논문에서 16이 계산 비용대비 효율적이라 소개한다. 

## 이때, p_h(p_w): 패치의 크기 and n_h(n_w): 패치의 크기 나눴을 때 가로 수 
patches = rearrange(x, 'b c (p_h n_h) (p_w n_w) -> b (n_h n_w) (p_h p_w c)', p_h=p, p_w=p)


########### 결과
# x : torch.Size([1000, 3, 224, 224])
# patches : torch.Size([1000, 196, 768])

In [25]:
from torchsummary import summary
import pandas as pd
import math 

in_channel = 3
patch_size = 16
embedding = pow(patch_size, 2) * in_channel

patch_layer = nn.Sequential(
    nn.Conv2d(in_channel, embedding, kernel_size= patch_size, stride= patch_size),
    Rearrange('b e h w -> b (h w) e')
).cuda()
##  input을 여러 개 받는 다면 summary(model, [(1, 16, 16), (1, 28, 28)]) 이렇게 리스트로 담아주자
summary(patch_layer,  x.shape[1:], device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
         Rearrange-2             [-1, 196, 768]               0
Total params: 590,592
Trainable params: 590,592
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 2.30
Params size (MB): 2.25
Estimated Total Size (MB): 5.12
----------------------------------------------------------------


### Add Class token and Position Encoding 

In [ ]:
# (여담이지만, 개인적으로 class token 없어도 학습이 잘 이루어지는걸 확인했는데, 모델의 마지막 layer인 linear layer에서 모델의 모든 feature들이 계산되는걸 보면 굳이 없어도 되지 않은가 싶었다.)
# patch 당 Class token이 붙는다. (왜 사진 당 클래스 토큰을 안붙였지? 내가 아는 클래스가 아닌 것인가...??)


In [10]:
import torch
from torch import nn
from torch import Tensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange

x = torch.randn(1000, 3, 224, 224)

## Image를 이미지의 크기/패치의 크기의 패치로 쪼개고 Flatten 시킨다. 
p = 16 # 논문에서 16이 계산 비용대비 효율적이라 소개한다. 

## 이때, p_h(p_w): 패치의 크기 and n_h(n_w): 패치의 크기 나눴을 때 가로 수 
patches = rearrange(x, 'b c (p_h n_h) (p_w n_w) -> b (n_h n_w) (p_h p_w c)', p_h=p, p_w=p)

class PatchEmbedding(nn.Module):

    # defalut는 Paper를 따라간다.
    def __init__(self, in_channel: int = 3, patch_size: int = 16, embedding: int = 768, img_size: int = 224):
        super().__init__()
        self.patch_num = (img_size//patch_size)**2
        self.in_channel = in_channel
        self.patch_size = patch_size
        self.embedding = embedding

        # nn.Parameter는 PyTorch에서 모델의 '학습 가능한 파라미터'를 정의하는 데 사용되는 클래스입니다. 
        self.cls_token = nn.Parameter(torch.randn(1,1,self.embedding))

        # Position embedding에서 +1이 되는 이유는 cls token 때문이다.
        # 그냥 randn으로 Position encoding을 해도 되는가..?? 
        self.postions = nn.Parameter(torch.randn(self.patch_num+1, self.embedding))

        self.projection = nn.Sequential(
            # con2d말고도 Linear로 출력값을 맞출 수 있다. 하지만, con2d가 성능이 더 좋다고 말한다.
            nn.Conv2d(in_channels= self.in_channel, out_channels= self.embedding, kernel_size=self.patch_size, stride=self.patch_size),
            Rearrange('b e h w -> b (h w) e'), # 변환하는게 약하다. 보완하기!!!!!!!!!!!!!
        )
        # Linear 층이다.
        # self.projection = nn.Sequential(
        #     # break-down the image in s1 x s2 patches and flat them
        #     Rearrange('b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=patch_size, s2=patch_size),
        #     nn.Linear(patch_size * patch_size * in_channels, emb_size)
        # )
    
    def forward(self, x: Tensor) -> Tensor:
        x = self.projection(x)
        #### Add cls token 
        # class token의 shape은 (1, 1, embedding) 
        # 이는 사진 한개 당 embedding 차원의 학습파라미터로 훈련을 진행한다. 
        cls_tokens = repeat(self.cls_token,'() n e -> b n e', b = x.shape[0])
        # print(x.shape) # torch.Size([1000, 196, 768])
        x = torch.cat([cls_tokens, x], dim=1)
        # print(x.shape) # torch.Size([1000, 197, 768])
        
        #### Add Position Embedding  
        print(self.postions.shape)
        # 텐서끼리 더할때 행과 열이 일치한다면 예상과 동일하게 더하기를 진행한다. 
        x += self.postions 
        return x

x= PatchEmbedding(x)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [8]:
result[0][0][0].sum()

tensor(1.0000)